In [1]:
# Importando bibliotecas
import pandas as pd 
import numpy as np 

import requests
from bs4 import BeautifulSoup as bs

import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [2]:
URL = 'https://www.vivareal.com.br/venda/distrito-federal/brasilia/bairros/setor-sudoeste/apartamento_residencial/?pagina={}'
i = 1
retorno = requests.get(URL)
soup = bs(retorno.text)

###### **Verificando o tamanho de objetos carregados por página**

In [3]:
objetos_pag = soup.find_all('article', {'class':'property-card__container js-property-card'})
len(objetos_pag)

36

###### **Obtendo o primeiro objeto e verificando os atributos que serão importante e a classe ao qual pertence**

In [4]:
objeto = objetos_pag[0]
objeto

<article class="property-card__container js-property-card" data-see-phone=""> <div class="property-card__main-info"> <div class="property-card__main-link"> <div class="property-card__carousel js-property-carousel"> <a class="property-card__labels-container js-main-info js-listing-labels-link" href="/imovel/apartamento-4-quartos-setor-sudoeste-bairros-brasilia-com-garagem-146m2-venda-RS2250000-id-2542005292/"> <div class="property-card__inactive-listing">Indisponível</div> <div class="property-card__already-seen">Visualizado</div> </a> <div class="carousel__container js-carousel-scroll" style="width: calc(5 * 100%);"> <div class="carousel__item-wrapper js-carousel-item-wrapper"> <img alt="Foto 1 de Apartamento com 4 Quartos à venda, 146m² em Setor Sudoeste, Brasília" class="carousel__image js-carousel-image" src="https://resizedimgs.vivareal.com/crop/286x200/vr.images.sp/b53ff5454ff66192c2aeb0a879fa382c.jpg" title="Apartamento com 4 Quartos à venda, 146m² em Setor Sudoeste, Brasília - i

###### **Colocando cada objeto dentro de uma variável**

In [5]:
qtd_imoveis = float(soup.find('strong', {'class':'results-summary__count js-total-records'}).text.replace('.',''))
descricao = soup.find('span', {'class': 'property-card__title js-cardLink js-card-title'}).text.strip()
endereco = soup.find('span', {'class': 'property-card__address'}).text.strip()
metragem = int(soup.find('li', {'class': 'property-card__detail-item property-card__detail-area'}).span.text.strip())
quartos = int(soup.find('li', {'class': 'property-card__detail-room'}).span.text.strip())
banheiro = int(soup.find('li',{'class': 'property-card__detail-bathroom'}).span.text.strip())
vagas = int(soup.find('li',{'class': 'property-card__detail-garage'}).span.text.strip())
valor = soup.find(class_='property-card__price js-property-card-prices js-property-card__price-small').text.strip()
condominio = soup.find('strong', {'class':'js-condo-price'}).text.strip()

print(qtd_imoveis)
print(descricao)
print(endereco)
print(metragem)
print(quartos)
print(banheiro)
print(vagas)
print(valor)
print(condominio)

68.0
Apartamento com 4 Quartos à Venda, 146m²
SQSW 300 Bloco B, 506 - Setor Sudoeste, Brasília - DF
146
4
5
2
R$ 2.250.000
R$ 1.200


###### **Criando Dataframe para organizar as variáveis encontradas. Deixei a quantidade de imóveis encontrados na pesquisa(64) fora do Dataframe**

In [6]:
df = pd.DataFrame(columns=[
    'descricao',
    'endereco',
    'metragem',
    'quartos',
    'banheiro',
    'vagas',
    'valor',
    'condominio',
])

i = 0

In [7]:
df.columns

Index(['descricao', 'endereco', 'metragem', 'quartos', 'banheiro', 'vagas',
       'valor', 'condominio'],
      dtype='object')

###### **Laço de repetição para inserir os 64 objetos encontrados**

In [8]:

while qtd_imoveis > df.shape[0]:
    
    i += 1
    print(f"valor i: {i} \t\t qtd_imoveis: {df.shape[0]}")
    ret = requests.get(URL.format(i))
    soup = bs(ret.text)

    final = soup.find_all('article', {'class':'property-card__container js-property-card'})

    for objeto in final:
        try:
            descricao = objeto.find('span', {'class': 'property-card__title js-cardLink js-card-title'}).text.strip()
        except: 
            descricao = None
        try:
            endereco = objeto.find('span', {'class': 'property-card__address'}).text.strip()
        except: 
            endereco = None
        try:
            metragem = int(objeto.find('li', {'class': 'property-card__detail-item property-card__detail-area'}).span.text.strip())
        except:
            metragem = None
        try:
            quartos = int(objeto.find('li', {'class': 'property-card__detail-room'}).span.text.strip())
        except: 
            quartos = None
        try: 
            banheiro = int(objeto.find('li',{'class': 'property-card__detail-bathroom'}).span.text.strip())
        except:
            banheiro = None
        try:
            vagas = int(objeto.find('li',{'class': 'property-card__detail-garage'}).span.text.strip())
        except:
            vagas = None
        try:
            valor = objeto.find(class_='property-card__price js-property-card-prices js-property-card__price-small').text.strip()
        except:
            valor = None
        try:
            condominio = objeto.find('strong', {'class':'js-condo-price'}).text.strip()
        except:
            condominio = None
            
        df.loc[df.shape[0]] = [
            descricao,
            endereco,
            metragem,
            quartos,
            banheiro,
            vagas,
            valor,
            condominio
        ]      

valor i: 1 		 qtd_imoveis: 0
valor i: 2 		 qtd_imoveis: 36


In [9]:
df.head(20)

,descricao,endereco,metragem,quartos,banheiro,vagas,valor,condominio
0,"Apartamento com 4 Quartos à Venda, 146m²","SQSW 300 Bloco B, 506 - Setor Sudoeste, Brasíl...",146,4,5,2,R$ 2.250.000,R$ 1.200
1,Haus by Novka,"Parque Recreativo Sarah Kubitschek, - - Asa Su...",None,None,None,1,Sob Consulta,None
2,"Apartamento com 2 Quartos à Venda, 90m²","SQSW 101 Bloco H, 101 - Setor Sudoeste, Brasíl...",90,2,2,1,R$ 1.300.000,None
3,"Apartamento com 3 Quartos à Venda, 90m²","SQSW 100 Bloco F, 603 - Setor Sudoeste, Brasíl...",90,3,1,1,R$ 1.165.000,R$ 800.000
4,"Apartamento com 3 Quartos à Venda, 90m²","SQSW 100 Bloco D, 1 - Setor Sudoeste, Brasília...",90,3,3,1,R$ 1.255.000,R$ 820
5,"Apartamento com 3 Quartos à Venda, 145m²","SQSW 306 Bloco F, 01 - Setor Sudoeste, Brasíli...",145,3,3,2,R$ 1.900.000,R$ 1.290
6,"Apartamento com 2 Quartos à Venda, 82m²","CCSW 4 Lote 5 Bloco B, 306 - Setor Sudoeste, B...",82,2,1,2,R$ 1.050.000,R$ 733
7,"Apartamento com Quarto à Venda, 57m²","QRSW 2 Bloco A-10, 10 - Setor Sudoeste, Brasíl...",57,1,1,None,R$ 530.000,None
8,Parque dos Cedros,"SQNW 306 Bloco A, s/n - Setor Noroeste, Brasíl...",165,4,6,4,R$ 2.671.312,None
9,"Apartamento com 4 Quartos à Venda, 144m²","SQSW 500, 5 - Setor Sudoeste, Brasília - DF",144,4,5,3,R$ 2.115.314,None


###### **Limpeza dos dados.**

In [10]:

df['valor'] = df['valor'].apply(lambda x: x.replace('R$','').strip())
df['condominio'] = df['condominio'].replace(np.nan, 'R$ 0.0')
df['condominio'] = df['condominio'].apply(lambda x: x.replace('R$','').strip())
#df['condominio'] = df['condominio'].apply(lambda x: x.replace('R$','').strip())

df.head(2)

,descricao,endereco,metragem,quartos,banheiro,vagas,valor,condominio
0,"Apartamento com 4 Quartos à Venda, 146m²","SQSW 300 Bloco B, 506 - Setor Sudoeste, Brasíl...",146,4,5,2,2.250.000,1.200
1,Haus by Novka,"Parque Recreativo Sarah Kubitschek, - - Asa Su...",None,None,None,1,Sob Consulta,0.0


In [11]:
df.dtypes

descricao     object
endereco      object
metragem      object
quartos       object
banheiro      object
vagas         object
valor         object
condominio    object
dtype: object

###### **Convertendo os tipos de valor e condomínio para float.** 

In [12]:


df['valor'] = df['valor'].apply(lambda v: locale.atof(v.split()[-1]))
df['condominio'] = df['condominio'].apply(lambda v: locale.atof(v.split()[-1]))

df.head(2)

ValueError: could not convert string to float: 'Consulta'

In [ ]:
df.to_csv('imoveis.csv', sep=',', index=False)